### Random Sampling Validation and Dataset Creation!

<br> 

##### In this notebook we aim to
- Count proportion of imbalanced features from each year (global truth)
- Speedily and randomly select from sql database
- Count proportion of imbalanced features from each year and each random sample (sampled truth)
- Compare global truth and sampled truth and deem acceptibility

<br>

#### Steps to getting there:
- import yearly database tables
    - get count of imbalanced features
    - calculate the ratio
    - store results
- import random subsample of yearly database
    - repeat above
- compare the global and local proportions graphically
- after sampling is acceptable then save the csv

### Import relevant libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import seaborn as sns
from matplotlib.ticker import PercentFormatter
import numpy as np

### Creating connection to local sqlite database

In [3]:
# !pip install ipython-sql
%load_ext sql
%sql sqlite:////Users/michaellink/Desktop/__NYCDSA/_Projects/Capstone/data/citibike/sqlite/citi_bike_yr.db

### Comparing the global 'truth' and local 'representation' of imbalanced features

#### Global - Proportion of rides taken from each station

In [ ]:
%%time
# sql query for total rides from station
result = %sql SELECT start_station_name, SUM (counter) from yr_2019 GROUP BY start_station_name;
# convert into pandas df
df_station_global = result.DataFrame()
# sort descending
df_station_global.sort_values(by=['SUM (counter)'], inplace=True, ascending=False)
# rename column
df_station_global.rename(columns={"SUM (counter)": "num_rides"}, inplace=True)
# getting proportion of rides from each station
df_station_global['percent_rides'] = 100 * (df_station_global['num_rides'] / df_station_global['num_rides'].sum())
# displaying results
df_station_global.head(2)

#### Local - Proportion of rides taken from each station

In [ ]:
%%time
# sql query to get 5% random sample
result = %sql SELECT * FROM yr_2019 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.05;
# convert into pandas df
df_station_local = result.DataFrame()
# group by station name and sum occurences
df_station_local = df_station_local.groupby(['start_station_name'])['counter'].sum().reset_index()
# sort descending
df_station_local.sort_values(by=['counter'], inplace=True, ascending=False)
# rename column
df_station_local.rename(columns={"counter": "num_rides"}, inplace=True)
# getting proportion of rides from each station
df_station_local['percent_rides'] = 100 * (df_station_local['num_rides'] / df_station_local['num_rides'].sum())
# displaying results
df_station_local.head(2)
df_station_local.tail(10)

# Size of subsampled csv is 50kb
# df_station_local.to_csv('size.csv') 

#### Function to perform sample validation (i.e. is our sample representative?)

In [ ]:
def sample_validation(df_station_global = df_station_global,
                      df_station_local = df_station_local,
                      num_samples = 3,
                      ratio_grab = 0.05):
    
    # Prepare data FOR the LOOP
    df_station_global_for_loop = df_station_global.drop(['num_rides'], axis=1).rename(columns={"percent_rides": "percent_global_rides"}).set_index('start_station_name')
    
    # Perform first concatenation
    # sql query to get 5% random sample
    result = %sql SELECT * FROM yr_2019 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < $ratio_grab;
    # convert into pandas df
    df_station_local = result.DataFrame()
    # group by station name and sum occurences
    df_station_local = df_station_local.groupby(['start_station_name'])['counter'].sum().reset_index()
    # sort descending
    df_station_local.sort_values(by=['counter'], inplace=True, ascending=False)
    # rename column
    df_station_local.rename(columns={"counter": "num_rides"}, inplace=True)
    # getting proportion of rides from each station
    df_station_local['percent_rides'] = 100 * (df_station_local['num_rides'] / df_station_local['num_rides'].sum())
    # modify df_station_local to prepare for concatenation
    df_station_local_for_loop = df_station_local.drop(['num_rides'], axis=1).set_index('start_station_name')
    
    # Do initial concatenation
    df_station_sampling = pd.concat([df_station_global_for_loop, df_station_local_for_loop], axis=1, sort=False).reindex(df_station_global_for_loop.index)

    # for loop to randomly sample from sql database then save results (proportion of rides taken from a station)
    for x in range(0, (num_samples - 1)):
        print(f'This is iteration {x}')
        # sql query to get 5% random sample
        result = %sql SELECT * FROM yr_2019 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < $ratio_grab;
        # convert into pandas df
        df_station_local_for_loop = result.DataFrame()
        # group by station name and sum occurences
        df_station_local_for_loop = df_station_local_for_loop.groupby(['start_station_name'])['counter'].sum().reset_index()
        # sort descending
        df_station_local_for_loop.sort_values(by=['counter'], inplace=True, ascending=False)
        # rename column
        df_station_local_for_loop.rename(columns={"counter": "num_rides"}, inplace=True)
        # getting proportion of rides from each station
        df_station_local_for_loop['percent_rides'] = 100 * (df_station_local_for_loop['num_rides'] / df_station_local_for_loop['num_rides'].sum())
        # modify df_station_local to prepare for concatenation
        df_station_local_for_loop = df_station_local_for_loop.drop(['num_rides'], axis=1).set_index('start_station_name')
        # concatenate this iteration of df_station_local
        df_station_sampling = pd.concat([df_station_sampling, df_station_local_for_loop], axis=1, sort=False).reindex(df_station_sampling.index)

    return df_station_sampling

In [ ]:
# Grab 5% of the data. Do this 40 times. Save Results
sample_five_percent_n_40 = sample_validation(num_samples = 40, ratio_grab = 0.05)

In [ ]:
sample_five_percent_n_40.to_csv('five_percent_sample.csv')

In [ ]:
# Grab 0.5% of the data. Do this 40 times. Save Results
sample_one_half_percent_n_40 = sample_validation(num_samples = 40, ratio_grab = 0.005)

In [ ]:
sample_one_half_percent_n_40.to_csv('one_half_percent_sample.csv')

In [ ]:
# Grab 15% of the data. Do this 40 times. Save Results
sample_fifteen_percent_n_40 = sample_validation(num_samples = 40, ratio_grab = 0.15)

In [ ]:
sample_fifteen_percent_n_40.to_csv('fifteen_percent_sample.csv')

### Conclusions

In [ ]:
sample_fifteen_percent_n_40 = pd.read_csv('fifteen_percent_sample.csv')
sample_five_percent_n_40 = pd.read_csv('five_percent_sample.csv')
sample_one_half_percent_n_40 = pd.read_csv('one_half_percent_sample.csv')

In [ ]:
sample_fifteen_percent_n_40.head(1)

In [ ]:
fifteen = sample_fifteen_percent_n_40.copy().set_index('start_station_name')
five = sample_five_percent_n_40.copy().set_index('start_station_name')
half = sample_one_half_percent_n_40.copy().set_index('start_station_name')

In [ ]:
num = pd.DataFrame(half.isna().sum()).iloc[1:][0].mean()
print(f'The average number of missed stations when using 1/2 % sampling is {num}.\n')
num = pd.DataFrame(five.isna().sum()).iloc[1:][0].mean()
print(f'The average number of missed stations when using 5 % sampling is {num}.\n')
num = pd.DataFrame(fifteen.isna().sum()).iloc[1:][0].mean()
print(f'The average number of missed stations when using 15 % sampling is {num}.\n')

In [ ]:
correct = fifteen.copy().transpose().iloc[:1]
fifteen = fifteen.copy().transpose().iloc[1:]
five = five.copy().transpose().iloc[1:]
half = half.copy().transpose().iloc[1:]

In [ ]:
correct.head(2)

In [ ]:
# assign random number so we can look at a variety of station distributions
wild = random.randrange(0, 39)

# get station name associated with this random column
station_name = correct.iloc[:,wild:wild+1].columns.to_list()[0]
# get the true distribution
correct_answer= correct.iloc[:,wild:wild+1].values.flatten()[0]

x = fifteen.copy().iloc[:,wild:wild+1].values.flatten()
plt.figure(figsize=(20,10)) 
result = plt.hist(x, bins=10, color='c', edgecolor='k', alpha=0.65)
plt.axvline(x.mean(), color='r', linestyle='dashed', linewidth=2, label='Sample Mean of % Rides = '+str(round(x.mean(),2)))
plt.axvline(correct_answer, color='g', linestyle='dashed', linewidth=2, label='Population % Rides = '+str(round(x.mean(),2)))
plt.title(station_name+' for the 15% Subsample Iterated 40 Times')
plt.xlabel('Percentage of Rides Taken From this Station in 2019')
plt.ylabel('Frequency')
plt.legend(loc='upper right')
plt.show()


In [ ]:
# assign random number so we can look at a variety of station distributions
wild = random.randrange(0, 990)

# get station name associated with this random column
station_name = correct.iloc[:,wild:wild+1].columns.to_list()[0]
# get the true distribution
correct_answer= correct.iloc[:,wild:wild+1].values.flatten()[0]

x = five.copy().iloc[:,wild:wild+1].values.flatten()
plt.figure(figsize=(20,10)) 
result = plt.hist(x, bins=10, color='c', edgecolor='k', alpha=0.65)
plt.axvline(x.mean(), color='r', linestyle='dashed', linewidth=2, label='Sample Mean of % Rides = '+str(round(x.mean(),2)))
plt.axvline(correct_answer, color='g', linestyle='dashed', linewidth=2, label='Population % Rides = '+str(round(x.mean(),2)))
plt.title(station_name+' for the 5% Subsample Iterated 40 Times')
plt.xlabel('Percentage of Rides Taken From this Station in 2019')
plt.ylabel('Frequency')
plt.legend(loc='upper right')
plt.show()


In [ ]:
df = fifteen.copy().iloc[:,6:7]
dfm = df.melt(var_name='columns')
g = sns.FacetGrid(dfm, col='columns')
g = (g.map(sns.distplot, 'value'))

In [ ]:

max_fifteen = fifteen.max().values.flatten()
min_fifteen = fifteen.min().values.flatten()
correct_values = correct.mean().values.flatten()
percent_off_max = 100 * ( (max_fifteen - correct_values) / correct_values)
weights = np.ones_like(correct_values) / len(correct_values)
plt.figure(figsize=(20,7)) 
result = plt.hist(percent_off_max, bins=25, color='c', edgecolor='k', alpha=0.65, weights=weights)
plt.xticks(np.arange(0, 360, 10))
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.title('Percent Correct From 15% Sample >>> 100 * ( Max - Correct / Correct )')
plt.xlabel('Percentage Deviation from Correct # of Rides Taken From this Station in 2019')
plt.ylabel('Percent of Samples Falling Within Bin')
plt.show()

In [ ]:

max_five = five.max().values.flatten()
min_five = five.min().values.flatten()
correct_values = correct.mean().values.flatten()
percent_off_max = 100 * ( (max_five - correct_values) / correct_values)
weights = np.ones_like(correct_values) / len(correct_values)
plt.figure(figsize=(20,10)) 
result = plt.hist(percent_off_max, bins=25, color='c', edgecolor='k', alpha=0.65, weights=weights)
# plt.xticks(np.arange(0, 360, 10))
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.title('Percent Correct From 5% Sample>>> 100 * ( Max - Correct / Correct )')
plt.show()

In [ ]:
min_fifteen = fifteen.min().values.flatten()
correct_values = correct.mean().values.flatten()
percent_off_min = 100 * ( (correct_values - min_fifteen) / correct_values)
weights = np.ones_like(correct_values) / len(correct_values)
plt.figure(figsize=(20,7)) 
result = plt.hist(percent_off_min, bins=25, color='c', edgecolor='k', alpha=0.65, weights=weights)
plt.xticks(np.arange(-130, 100, 10))
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.title('Percent Correct From 15% Sample >>> 100 * ( Correct - Min / Correct )')
plt.show()

In [ ]:
min_five = five.min().values.flatten()
correct_values = correct.mean().values.flatten()
percent_off_min = 100 * ( (correct_values - min_five) / correct_values)
weights = np.ones_like(correct_values) / len(correct_values)
plt.figure(figsize=(20,7)) 
result = plt.hist(percent_off_min, bins=25, color='c', edgecolor='k', alpha=0.65, weights=weights)
# plt.xticks(np.arange(-130, 100, 10))
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.title('Percent Correct From 5% Sample >>> 100 * ( Correct - Min / Correct )')
plt.show()

In [ ]:
max_five = five.max().values.flatten()
min_five = five.min().values.flatten()
correct_values = correct.mean().values.flatten()
percent_off_max = 100 * ( (max_five - correct_values) / correct_values)
percent_off_min = 100 * ( (correct_values - min_five) / correct_values)

med_max_diff = round(np.median(percent_off_max),2)
med_min_diff = round(np.median(percent_off_min),2)

print(f'The median percent difference for the 5% sample is {med_max_diff} when using the max and {med_min_diff} when using the min.\n')

max_fifteen = fifteen.max().values.flatten()
min_fifteen = fifteen.min().values.flatten()
correct_values = correct.mean().values.flatten()
percent_off_max = 100 * ( (max_fifteen - correct_values) / correct_values)
percent_off_min = 100 * ( (correct_values - min_fifteen) / correct_values)

med_max_diff = round(np.median(percent_off_max),2)
med_min_diff = round(np.median(percent_off_min),2)

print(f'The median percent difference for the 5% sample is {med_max_diff} when using the max and {med_min_diff} when using the min.\n')

### Data Export Using 15% Subsample

In [4]:
# sql query to get 10% random sample
result = %sql SELECT * FROM yr_2020 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.15;
# convert into pandas df
yr_2020 = result.DataFrame()
yr_2020.drop(columns=['dummy'], inplace=True)
# send to csv
yr_2020.to_csv('./rand_10/yr_2020.csv', index=False)
yr_2020.head(3)

 * sqlite:////Users/michaellink/Desktop/__NYCDSA/_Projects/Capstone/data/citibike/sqlite/citi_bike_yr.db
Done.


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,counter,year,age
0,114.0,2020-01-01 00:03:49.8380,2020-01-01 00:05:44.1390,274,Lafayette Ave & Fort Greene Pl,40.686919,-73.976682,416,Cumberland St & Lafayette Ave,40.687534,-73.972652,14936,Subscriber,1991,1,1,2020,29
1,1095.0,2020-01-01 00:05:36.6700,2020-01-01 00:23:52.0200,514,12 Ave & W 40 St,40.760875,-74.002777,405,Washington St & Gansevoort St,40.739323,-74.008119,30266,Customer,2001,2,1,2020,19
2,579.0,2020-01-01 00:05:59.9140,2020-01-01 00:15:39.6350,3349,Grand Army Plaza & Plaza St West,40.672968,-73.970880,3411,Bond St & Bergen St,40.684967,-73.986208,39925,Subscriber,1961,1,1,2020,59


In [5]:
del yr_2020

In [7]:
# sql query to get 10% random sample
result = %sql SELECT * FROM yr_2019 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.15;
# convert into pandas df
yr_2019 = result.DataFrame()
yr_2019.drop(columns=['dummy'], inplace=True)
# send to csv
yr_2019.to_csv('./rand_10/yr_2019.csv', index=False)
yr_2019.head(3)

 * sqlite:////Users/michaellink/Desktop/__NYCDSA/_Projects/Capstone/data/citibike/sqlite/citi_bike_yr.db
Done.


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,counter,year,age
0,505.0,2019-01-01 00:18:45.1180,2019-01-01 00:27:10.5990,3132,E 59 St & Madison Ave,40.763505,-73.971092,359,E 47 St & Park Ave,40.755103,-73.974987,31801,Subscriber,1981,1,1,2019,38
1,537.0,2019-01-01 00:21:42.7480,2019-01-01 00:30:40.4410,3231,E 67 St & Park Ave,40.767801,-73.965921,519,Pershing Square North,40.751873,-73.977706,30089,Subscriber,1996,1,1,2019,23
2,3491.0,2019-01-01 00:27:17.1280,2019-01-01 01:25:28.2510,3574,Prospect Pl & Underhill Ave,40.676969,-73.965790,432,E 7 St & Avenue A,40.726218,-73.983799,32404,Subscriber,1994,1,1,2019,25


In [8]:
del yr_2019

In [9]:
# sql query to get 10% random sample
result = %sql SELECT * FROM yr_2018 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.15;
# convert into pandas df
yr_2018 = result.DataFrame()
yr_2018.drop(columns=['dummy'], inplace=True)
# send to csv
yr_2018.to_csv('./rand_10/yr_2018.csv', index=False)
yr_2018.head(3)

 * sqlite:////Users/michaellink/Desktop/__NYCDSA/_Projects/Capstone/data/citibike/sqlite/citi_bike_yr.db
Done.


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,counter,year,age
0,306.0,2018-01-01 18:14:51.5680,2018-01-01 18:19:57.6420,72,W 52 St & 11 Ave,40.767272,-73.993929,3356,Amsterdam Ave & W 66 St,40.774667,-73.984706,30319,Subscriber,1992,1,1,2018,26
1,434.0,2018-01-02 08:06:14.3310,2018-01-02 08:13:28.7670,72,W 52 St & 11 Ave,40.767272,-73.993929,173,Broadway & W 49 St,40.760683,-73.984527,30525,Subscriber,1983,1,1,2018,35
2,1200.0,2018-01-02 10:23:18.8800,2018-01-02 10:43:18.9110,72,W 52 St & 11 Ave,40.767272,-73.993929,127,Barrow St & Hudson St,40.731724,-74.006744,30257,Subscriber,1974,1,1,2018,44


In [10]:
del yr_2018

In [11]:
# sql query to get 10% random sample
result = %sql SELECT * FROM yr_2017 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.15;
# convert into pandas df
yr_2017 = result.DataFrame()
yr_2017.drop(columns=['dummy'], inplace=True)
# send to csv
yr_2017.to_csv('./rand_10/yr_2017.csv', index=False)
yr_2017.head(3)

 * sqlite:////Users/michaellink/Desktop/__NYCDSA/_Projects/Capstone/data/citibike/sqlite/citi_bike_yr.db
Done.


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,counter,year,age
0,631.0,2017-01-01 00:01:10,2017-01-01 00:11:42,3143,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,40.768737,-73.961199,21211,Customer,,0,1,2017,2017
1,559.0,2017-01-01 00:05:00,2017-01-01 00:14:20,499,Broadway & W 60 St,40.769155,-73.981918,479,9 Ave & W 45 St,40.760193,-73.991255,27294,Subscriber,1973,1,1,2017,44
2,1504.0,2017-01-01 00:09:29,2017-01-01 00:34:34,423,W 54 St & 9 Ave,40.765849,-73.986905,3263,Cooper Square & E 7 St,40.729236,-73.990868,17810,Subscriber,1994,1,1,2017,23


In [12]:
del yr_2017

In [13]:
# sql query to get 10% random sample
result = %sql SELECT * FROM yr_2016 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.15;
# convert into pandas df
yr_2016 = result.DataFrame()
yr_2016.drop(columns=['dummy'], inplace=True)
# send to csv
yr_2016.to_csv('./rand_10/yr_2016.csv', index=False)
yr_2016.head(3)

 * sqlite:////Users/michaellink/Desktop/__NYCDSA/_Projects/Capstone/data/citibike/sqlite/citi_bike_yr.db
Done.


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,counter,year,age
0,889.0,1/1/2016 00:01:06,1/1/2016 00:15:56,268,Howard St & Centre St,40.719105,-73.999733,3002,South End Ave & Liberty St,40.711512,-74.015756,22794,Subscriber,1961,2,1,2016,55
1,1480.0,1/1/2016 00:01:12,1/1/2016 00:25:52,2006,Central Park S & 6 Ave,40.765909,-73.976342,2006,Central Park S & 6 Ave,40.765909,-73.976342,14562,Subscriber,1952,1,1,2016,64
2,479.0,1/1/2016 00:08:42,1/1/2016 00:16:42,3117,Franklin St & Dupont St,40.735640,-73.958660,3104,Kent Ave & N 7 St,40.720577,-73.961502,23563,Subscriber,1993,1,1,2016,23


In [14]:
del yr_2016

In [15]:
# sql query to get 10% random sample
result = %sql SELECT * FROM yr_2015 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.15;
# convert into pandas df
yr_2015 = result.DataFrame()
yr_2015.drop(columns=['dummy'], inplace=True)
# send to csv
yr_2015.to_csv('./rand_10/yr_2015.csv', index=False)
yr_2015.head(3)

 * sqlite:////Users/michaellink/Desktop/__NYCDSA/_Projects/Capstone/data/citibike/sqlite/citi_bike_yr.db
Done.


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,counter,year,age
0,496.0,1/1/2015 0:07,1/1/2015 0:15,512,W 29 St & 9 Ave,40.750073,-73.998393,383,Greenwich Ave & Charles St,40.735238,-74.000271,20788,Subscriber,1969,2,1,2015,46
1,540.0,1/1/2015 0:10,1/1/2015 0:19,291,Madison St & Montgomery St,40.713126,-73.984844,151,Cleveland Pl & Spring St,40.721816,-73.997203,16947,Subscriber,1979,1,1,2015,36
2,1196.0,1/1/2015 0:16,1/1/2015 0:36,466,W 25 St & 6 Ave,40.743954,-73.991449,454,E 51 St & 1 Ave,40.754557,-73.965930,20683,Subscriber,1971,1,1,2015,44


In [16]:
del yr_2015

In [17]:
# sql query to get 10% random sample
result = %sql SELECT * FROM yr_2014 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.15;
# convert into pandas df
yr_2014 = result.DataFrame()
yr_2014.drop(columns=['dummy'], inplace=True)
# send to csv
yr_2014.to_csv('./rand_10/yr_2014.csv', index=False)
yr_2014.head(3)

 * sqlite:////Users/michaellink/Desktop/__NYCDSA/_Projects/Capstone/data/citibike/sqlite/citi_bike_yr.db
Done.


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,counter,year,age
0,330.0,2014-01-01 00:05:55,2014-01-01 00:11:25,422,W 59 St & 10 Ave,40.770513,-73.988038,526,E 33 St & 5 Ave,40.747659,-73.984907,17343,Subscriber,1987,1,1,2014,27
1,718.0,2014-01-01 00:09:32,2014-01-01 00:21:30,263,Elizabeth St & Hester St,40.717290,-73.996375,251,Mott St & Prince St,40.723180,-73.994800,15693,Customer,\N,0,1,2014,2014
2,744.0,2014-01-01 00:12:23,2014-01-01 00:24:47,450,W 49 St & 8 Ave,40.762272,-73.987882,505,6 Ave & W 33 St,40.749013,-73.988484,15157,Subscriber,1976,1,1,2014,38


In [18]:
del yr_2014

In [19]:
# sql query to get 10% random sample
result = %sql SELECT * FROM yr_2013 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.15;
# convert into pandas df
yr_2013 = result.DataFrame()
yr_2013.drop(columns=['dummy'], inplace=True)
# send to csv
yr_2013.to_csv('./rand_10/yr_2013.csv', index=False)
yr_2013.head(3)

 * sqlite:////Users/michaellink/Desktop/__NYCDSA/_Projects/Capstone/data/citibike/sqlite/citi_bike_yr.db
Done.


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,counter,year,age
0,456.0,2013-07-01 00:02:23,2013-07-01 00:09:59,146,Hudson St & Reade St,40.716250,-74.009106,351,Front St & Maiden Ln,40.7053,-74.0061,16321,Subscriber,1976,1,1,2013,37
1,311.0,2013-07-01 00:03:58,2013-07-01 00:09:09,494,W 26 St & 8 Ave,40.747348,-73.997236,458,11 Ave & W 27 St,40.7514,-74.0052,20595,Customer,\N,0,1,2013,2013
2,318.0,2013-07-01 00:05:09,2013-07-01 00:10:27,406,Hicks St & Montague St,40.695128,-73.995951,237,E 11 St & 2 Ave,40.7305,-73.9867,19170,Subscriber,1983,2,1,2013,30


In [20]:
del yr_2013

In [ ]:
# sql query to get 5% random sample
result = %sql SELECT * FROM yr_2020 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.05;
# convert into pandas df
yr_2020 = result.DataFrame()
yr_2020.drop(columns=['dummy'], inplace=True)
# send to csv
yr_2020.to_csv('./rand_5/yr_2020.csv', index=False)
yr_2020.sample(5)

In [ ]:
# sql query to get 5% random sample
result = %sql SELECT * FROM yr_2019 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.05;
# convert into pandas df
yr_2019 = result.DataFrame()
yr_2019.drop(columns=['dummy'], inplace=True)
# # send to csv
yr_2019.to_csv('./rand_5/yr_2019.csv', index=False)
yr_2019.head()

In [ ]:
# sql query to get 5% random sample
result = %sql SELECT * FROM yr_2018 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.05;
# convert into pandas df
yr_2018 = result.DataFrame()
yr_2018.drop(columns=['dummy'], inplace=True)
# send to csv
yr_2018.to_csv('./rand_5/yr_2018.csv', index=False)

In [ ]:
# sql query to get 5% random sample
result = %sql SELECT * FROM yr_2017 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.05;
# convert into pandas df
yr_2017 = result.DataFrame()
yr_2017.drop(columns=['dummy'], inplace=True)
# send to csv
yr_2017.to_csv('./rand_5/yr_2017.csv', index=False)

In [ ]:
# sql query to get 5% random sample
result = %sql SELECT * FROM yr_2016 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.05;
# convert into pandas df
yr_2016 = result.DataFrame()
yr_2016.drop(columns=['dummy'], inplace=True)
# send to csv
yr_2016.to_csv('./rand_5/yr_2016.csv', index=False)

In [ ]:
# sql query to get 5% random sample
result = %sql SELECT * FROM yr_2015 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.05;
# convert into pandas df
yr_2015 = result.DataFrame()
yr_2015.drop(columns=['dummy'], inplace=True)
# send to csv
yr_2015.to_csv('./rand_5/yr_2015.csv', index=False)

In [ ]:
# sql query to get 5% random sample
result = %sql SELECT * FROM yr_2014 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.05;
# convert into pandas df
yr_2014 = result.DataFrame()
yr_2014.drop(columns=['dummy'], inplace=True)
# send to csv
yr_2014.to_csv('./rand_5/yr_2014.csv', index=False)

In [ ]:
# sql query to get 5% random sample
result = %sql SELECT * FROM yr_2013 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.05;
# convert into pandas df
yr_2013 = result.DataFrame()
yr_2013.drop(columns=['dummy'], inplace=True)
# send to csv
yr_2013.to_csv('./rand_5/yr_2013.csv', index=False)

In [ ]:
yr_2020.head(2)

In [ ]:
yr_2019.head(2)

In [ ]:
yr_2018.head(2)

In [ ]:
yr_2017.head(2)

In [ ]:
yr_2016.head(2)

In [ ]:
yr_2015.head(2)

In [ ]:
yr_2014.head(2)

In [ ]:
yr_2013.head(2)

In [ ]:
# convert into pandas df
yr_2019.head()

In [ ]:
df_2019 = pd.read_csv('/Users/michaellink/Desktop/__NYCDSA/_Projects/Capstone/scripts/workspace/Link_Michael/tripdata/_2019/combined.csv', nrows=10)

In [ ]:
df_2019.head()

#### Global - Total number of stations

In [ ]:
# get total number of distinct stations
result = %sql SELECT COUNT (DISTINCT start_station_name) FROM yr_2019;
num_station_global = result.DataFrame()
num_station_global.rename(columns={"COUNT (DISTINCT start_station_name)": "value"}, inplace=True)
num_station_global = num_station_global['value'].iloc[0]
num_station_global

#### Local - Total number of stations

In [ ]:
%%time
# get total number of distinct stations
# %sql SELECT COUNT (DISTINCT start_station_name) FROM yr_2019
result = %sql SELECT COUNT (DISTINCT start_station_name) FROM yr_2019 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.05;
    # can replace with pd dataframe later
num_station_local = result.DataFrame()
num_station_local.rename(columns={"COUNT (DISTINCT start_station_name)": "value"}, inplace=True)
num_station_local = num_station_local['value'].iloc[0]
num_station_local

#### Local - Experimentation to choose random sample method and to validate 5% grab

In [ ]:
%%time
# [55 seconds]
# Test on 5% grab of 2019 data with order by method
# https://stackoverflow.com/questions/4114940/select-random-rows-in-sqlite
result = %sql select * from yr_2019 order by random() limit 1048862;

In [ ]:
%%time
# [17 seconds]
# Test on 5% grab of 2019 data with abs method
# https://stackoverflow.com/questions/4114940/select-random-rows-in-sqlite
result = %sql SELECT * FROM yr_2019 WHERE abs(CAST(random() AS REAL))/9223372036854775808 < 0.05;

In [ ]:
%%time
# [5 seconds]
# Test on converting 5% grab of 2019 data into pandas dataframe
# This contributes little to the overall time
df_random_method_2 = result.DataFrame()
print(df_random_method_2.shape)
df_random_method_2.head(2)

In [ ]:
# Verifying that our grab was truly 5%

# Get total number of rows
result = %sql SELECT COUNT(*) FROM yr_2019;
num_observations = result.DataFrame()
num_observations.rename(columns={"COUNT(*)": "value"}, inplace=True)
num_observations = num_observations['value'].iloc[0]

# Get number of rows from random sample
num_row = df_random_method_2.shape[0]  

# Calculate percentage
percent = round(100 * ( num_row / num_observations), 2)
print(f'The randomly sampled dataset is {percent} percent of the original dataset')

In [ ]:
%%time
result = %sql SELECT * FROM yr_2020_test WHERE ID IN (SELECT ID FROM yr_2019 ORDER BY RANDOM() LIMIT 100)
# select * from yr_2019 where random() <= .01;
df_random = result.DataFrame()
df_random.head(2)

In [ ]:
result = %sql SELECT * from yr_2019 WHERE random()<200 / (SELECT COUNT(1) from logs)::float ORDER BY random() LIMIT 100;
# select * from yr_2019 where random() <= .01;
df_random = result.DataFrame()
df_random.head(2)

In [ ]:
%%time
# Order database randomly and take top %
result = %sql select * from yr_2019 where random() <= .01;
df_random = result.DataFrame()
df_random.head(2)

In [ ]:
%alias_magic t timeit

In [ ]:
# Approximately 20 seconds. Not a big problem if we run once and save as csv
results = %sql select * from yr_2019 order by random() limit 10;
df_random = result.DataFrame()
df_random.head(5)